In [5]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,GlobalAveragePooling2D,AveragePooling2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
import streamlit as st
import numpy as np

2025-07-06 12:48:57.039154: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-06 12:48:57.102454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751806137.129188   25236 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751806137.137498   25236 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751806137.181562   25236 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
train_dir = '/home/vignesh/tomato_data/train'
test_dir = '/home/vignesh/tomato_data/val'

In [6]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=32)

train_data = train_data.map(lambda x, y: (x / 255.0, y))

Found 10000 files belonging to 10 classes.


I0000 00:00:1751806164.212654   25236 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=32)

val_data = val_data.map(lambda x, y: (x / 255.0, y))

Found 1000 files belonging to 10 classes.


In [10]:
conv_base = DenseNet121(
    weights='imagenet',
    include_top = False,
    input_shape=(256,256,3),
    pooling='avg'
)
conv_base.trainable = False

In [11]:
model = Sequential()
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())
model.add(Dense(120, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train_data, epochs=100, validation_data=val_data, callbacks=[EarlyStopping(patience=0)])

Epoch 1/100


I0000 00:00:1751796995.070491     863 service.cc:152] XLA service 0x7993ec003f80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751796995.070528     863 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-07-06 10:16:35.423405: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751796997.232892     863 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/313 ━━━━━━━━━━━━━━━━━━━━ 2:08:57 25s/step - accuracy: 0.0312 - loss: 3.0391

I0000 00:00:1751797012.354546     863 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 74s 157ms/step - accuracy: 0.3646 - loss: 1.9286 - val_accuracy: 0.7460 - val_loss: 0.9393
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.7581 - loss: 0.7851 - val_accuracy: 0.8330 - val_loss: 0.5399
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8343 - loss: 0.5289 - val_accuracy: 0.8670 - val_loss: 0.4098
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8754 - loss: 0.4017 - val_accuracy: 0.8860 - val_loss: 0.3435
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.9022 - loss: 0.3274 - val_accuracy: 0.9010 - val_loss: 0.2966
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.9122 - loss: 0.2782 - val_accuracy: 0.9110 - val_loss: 0.2659
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.9265 - loss: 0.2365 - val_accuracy: 0.9130 - val_loss: 0.2420
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 62ms/step - accuracy: 0.9378 - loss: 0.2031 - val

In [15]:
# Evaluate the model on the validation data
evaluation = model.evaluate(val_data)

# Print the evaluation metrics
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 173ms/step - accuracy: 0.1035 - loss: 2.5314
Validation Loss: 2.525118827819824
Validation Accuracy: 0.10499999672174454


In [10]:
model.save('leaf_disease_model.h5')

In [12]:
model_json = model.to_json()
with open('leaf_disease_model_architecture.json', 'w') as f:
    f.write(model_json)
model.save_weights('leaf_disease_model.weights.h5')

In [13]:
loaded_model = tf.keras.models.load_model('leaf_disease_model.h5')

In [ ]:
import tensorflow as tf
import datetime
import os

# Load saved model
loaded_model = tf.keras.models.load_model('leaf_disease_model.h5')

# Create a new log directory for TensorBoard
log_dir = './logs/model_analysis/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
os.makedirs(log_dir, exist_ok=True)

# Create TensorBoard callback for model analysis
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch'
)

print("Model loaded successfully!")


2025-07-06 11:00:15.103523: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-06 11:00:15.191475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751799615.224736   10754 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751799615.235534   10754 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751799615.290402   10754 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Model loaded successfully!
TensorBoard logs will be saved to: ./logs/model_analysis/20250706-110022

To start TensorBoard, run in terminal:
tensorboard --logdir=./logs/model_analysis/20250706-110022

Then open http://localhost:6006 in your browser


In [8]:
%load_ext tensorboard
%tensorboard --logdir=./logs/model_analysis/val_eval_20250706-110721

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
loaded_model = tf.keras.models.load_model('leaf_disease_model.h5')

# Evaluate on validation data
val_evaluation = loaded_model.evaluate(val_data)
val_loss, val_accuracy = val_evaluation[0], val_evaluation[1]

# Log validation metrics to TensorBoard
import datetime
val_log_dir = './logs/model_analysis/val_eval_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
val_summary_writer = tf.summary.create_file_writer(val_log_dir)
with val_summary_writer.as_default():
    tf.summary.scalar('val_loss', val_loss, step=0)
    tf.summary.scalar('val_accuracy', val_accuracy, step=0)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation metrics logged to: {val_log_dir}")

I0000 00:00:1751800011.066753   11953 service.cc:152] XLA service 0x78a414002350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751800011.067079   11953 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-07-06 11:06:51.273350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751800014.924390   11953 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/32 ━━━━━━━━━━━━━━━━━━━━ 11:29 22s/step - accuracy: 0.9062 - loss: 0.2125

I0000 00:00:1751800029.449223   11953 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 393ms/step - accuracy: 0.9548 - loss: 0.1295
Validation Loss: 0.1444859504699707
Validation Accuracy: 0.9520000219345093
Validation metrics logged to: ./logs/model_analysis/val_eval_20250706-110721


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


val_dir = '/home/vignesh/tomato_data/val'

model = tf.keras.models.load_model('leaf_disease_model.h5')

val_data = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(256, 256),
    batch_size=32,
    shuffle=False  # Important: do not shuffle for correct label order!
)
val_data = val_data.map(lambda x, y: (x / 255.0, y))

# Get predictions and true labels
y_true = []
y_pred = []

for images, labels in val_data:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

Found 1000 files belonging to 10 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━